In [2]:
#import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import os
from collections import Counter

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

In [3]:
races = ['Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race']


In [14]:

combos = []
for voting_age in [0, 1]:
    for hispanic in [0, 1]:
        for race in range(63):
            combos.append([voting_age, hispanic, race])

#This method computes noisy population sizes for each (voting age Y/N)*(hispanic Y/N)*race combination using
# a geometric mechanism.
#-------------------------------------------------
#INPUT:
# dft: dataframe corresponding to a BLOCK
# ep: epsilon value for the county-wide mechanism
#-------------------------------------------------
#OUTPUT: 
# noisy_queries: numpy 3D array with dimensions (2, 2, 63)
#          the ith element is an integer representing the (noisy) population size for the category in combos[i]
def get_noisy_queries(dft,  ep):
    
    
    # here, we use epsilon = ep*0.12*0.5, as per Table 2.1 in https://www.nap.edu/read/25978/chapter/3#13
    geo = Geometric(epsilon=ep*0.12*0.5, sensitivity=1)
    
    countshape= 0
    queries = []
    noisy_queries = np.zeros(shape=(2, 2, 63))
    for voting_age in [0, 1]:
        for hispanic in [0, 1]:
            for race in range(63):
                queries.append([[voting_age, hispanic, race], 0])

    
    # for i in range(len(queries)):
    #   category = queries[i][0]
    #   count = num entries in dft with those values of voting age, hispanic, race
    #   queries[i][1] = count
    #   noisy_queries[voting_age][hispanic][race] = geo.randomise(count) 
    
    for i in range(0,len(queries)):
        #age
        ag = queries[i][0][0]
        if queries[i][0][0] == 0:
            count = dft[dft['age']<18]
        else:
            count = dft[dft['age']>=18]
        #hispanic
        hispan = queries[i][0][1]
        count = count[count['hispanic']==hispan]
        #race
        ra = queries[i][0][2]
        count = count[count['race']==ra]
    
        countshape += count.shape[0]
    
        count_num = count.shape[0]
        queries[i][1] = count_num
        randomized =  geo.randomise(count_num)
        noisy_queries[ag][hispan][ra] = randomized
        

    
    
    #iterate through all hispanic/race
    # combos and add their population size to race_histogram
    
    race_histogram = np.zeros(shape=(2, 63))
    
    # race_histogram: numpy 2D array with dimension (2, 63)
    #          for row i,
    #          column 0 contains the population size of non-Hispanic individuals with race i
    #          column 1 contains the population size of Hispanic individuals with race i
    for i in range(2):
        for j in range(63):
            race_histogram[i][j] = noisy_queries[0][i][j] + noisy_queries[1][i][j]
    
    return race_histogram




#This method computes a (noisy) histogram of population size by race for the specified county, 
# summing over all blocks in the county the output of get_noisy_queries.
#--------------------------------------------------------------------------------------------
#INPUT: 
# county: dataframe containing county data  
# ep: epsilon value for the county-wide mechanism
#--------------------------------------------------------------------------------------------
#OUTPUT:


In [15]:
def orig_hist(dft):
    
    queries = []
    noisy_queries = np.zeros(shape=(2, 2, 63))
    for voting_age in [0, 1]:
        for hispanic in [0, 1]:
            for race in range(63):
                queries.append([[voting_age, hispanic, race], 0])
    
    
    
    # for i in range(len(queries)):
    #   category = queries[i][0]
    #   count = num entries in dft with those values of voting age, hispanic, race
    #   queries[i][1] = count
    #   noisy_queries[voting_age][hispanic][race] = geo.randomise(count) 
    for i in range(0,len(queries)):
        #age
        ag = queries[i][0][0]
        if queries[i][0][0] == 0:
            count = dft[dft['age']<18]
        else:
            count = dft[dft['age']>=18]
        #hispanic
        hispan = queries[i][0][1]
        count = count[count['hispanic']==hispan]
        #race
        ra = queries[i][0][2]
        count = count[count['race']==ra]
    
    
        count_num = count.shape[0]
        queries[i][1] = count_num
        noisy_queries[ag][hispan][ra] = count_num

    race_histogram = np.zeros(shape=(2, 63))
    
    # race_histogram: numpy 2D array with dimension (2, 63)
    #          for row i,
    #          column 0 contains the population size of non-Hispanic individuals with race i
    #          column 1 contains the population size of Hispanic individuals with race i
    for i in range(2):
        for j in range(63):
            race_histogram[i][j] = noisy_queries[0][i][j] + noisy_queries[1][i][j]
    
    return race_histogram  
        
        
        

In [17]:
states = ['california', 'pennsylvania', 'newmexico', 'georgia', 'northdakota', 'hawaii', 'missouri', 'massachussets', 'vermont']
counties = ['alameda', 'armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']

j=0
epsilon = np.arange(.01, 10, .1, float)


for county in counties:
    print(county)
    df_first = create_block_data('../homemade_data/'+county+'.csv')
    orig_df = df_first.copy()
    for index, row in orig_df.iterrows():
        d = row['race']
        orig_df.at[index, 'race'] = races.index(d)
    epsilon = np.arange(.01, 10, .1, float)

    for e in epsilon:
        print(str(e)[0:4])
        for i in range(100):                
            df = df_first.copy()
            racehist = get_noisy_queries(orig_df, 10)
            df_final = pd.DataFrame(list(zip(racehist[0], racehist[1])), columns = ['Nonhispanic', 'Hispanic'])
            filename = "newdp_runs/"+county+"/dprun_"+str(e)[0:4]+"_"+str(i)+".csv"
            csv_orig_data = df_final.to_csv(filename, index = True)
    j+=1


    
    print(df)

alameda
0.01
0.11
0.21
0.31
0.41
0.51
0.61
0.71
0.81
0.91
1.01
1.11
1.21
1.31
1.41
1.51
1.61
1.71
1.81
1.91
2.01
2.11
2.21
2.31
2.41
2.51
2.61
2.71
2.81
2.91
3.01
3.11
3.21
3.31
3.41
3.51
3.61
3.71
3.81
3.91
4.01
4.11
4.21
4.31
4.41
4.51
4.61
4.71
4.81
4.91
5.01
5.11
5.21
5.31
5.41
5.51
5.61
5.71
5.81
5.91
6.01
6.11
6.21
6.31
6.41
6.51
6.61
6.71
6.81
6.91
7.01
7.11
7.21
7.31
7.41
7.51
7.61
7.71
7.81
7.91
8.01
8.11
8.21
8.31
8.41
8.51
8.61
8.71
8.81
8.91
9.01
9.11
9.21
9.31
9.41
9.51
9.61
9.71
9.81
9.91
        id  age sex                                               race  \
0        1   32   f         Total!!Population of one race!!White alone   
1        2   20   f         Total!!Population of one race!!White alone   
2        3   44   m         Total!!Population of one race!!White alone   
3        4   22   f         Total!!Population of one race!!White alone   
4        5   15   f         Total!!Population of one race!!White alone   
...    ...  ...  ..                             

2.61
2.71
2.81
2.91
3.01
3.11
3.21
3.31
3.41
3.51
3.61
3.71
3.81
3.91
4.01
4.11
4.21
4.31
4.41
4.51
4.61
4.71
4.81
4.91
5.01
5.11
5.21
5.31
5.41
5.51
5.61
5.71
5.81
5.91
6.01
6.11
6.21
6.31
6.41
6.51
6.61
6.71
6.81
6.91
7.01
7.11
7.21
7.31
7.41
7.51
7.61
7.71
7.81
7.91
8.01
8.11
8.21
8.31
8.41
8.51
8.61
8.71
8.81
8.91
9.01
9.11
9.21
9.31
9.41
9.51
9.61
9.71
9.81
9.91
      id  age sex                                               race  \
0      1    7   m         Total!!Population of one race!!White alone   
1      2   39   m         Total!!Population of one race!!White alone   
2      3   33   m         Total!!Population of one race!!White alone   
3      4    6   f         Total!!Population of one race!!White alone   
4      5   24   m         Total!!Population of one race!!White alone   
..   ...  ...  ..                                                ...   
975  976   39   m  Total!!Two or More Races!!Population of two ra...   
976  977   13   m  Total!!Two or More Races!!Populatio

5.41
5.51
5.61
5.71
5.81
5.91
6.01
6.11
6.21
6.31
6.41
6.51
6.61
6.71
6.81
6.91
7.01
7.11
7.21
7.31
7.41
7.51
7.61
7.71
7.81
7.91
8.01
8.11
8.21
8.31
8.41
8.51
8.61
8.71
8.81
8.91
9.01
9.11
9.21
9.31
9.41
9.51
9.61
9.71
9.81
9.91
        id  age sex                                               race  \
0        1   32   f         Total!!Population of one race!!White alone   
1        2   57   m         Total!!Population of one race!!White alone   
2        3   65   f         Total!!Population of one race!!White alone   
3        4   24   m         Total!!Population of one race!!White alone   
4        5   19   m         Total!!Population of one race!!White alone   
...    ...  ...  ..                                                ...   
1085  1086   26   m  Total!!Two or More Races!!Population of two ra...   
1086  1087   55   m  Total!!Two or More Races!!Population of two ra...   
1087  1088   17   m  Total!!Two or More Races!!Population of two ra...   
1088  1089   49   f  Total!!Tw